In [1]:
import os 
import string
import re
from pathlib import Path
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 
pd.options.mode.chained_assignment = None # no warnings
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
# tqdm
from tqdm import tqdm
# nltk
import nltk
from nltk.corpus import stopwords
# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity # to calculate the similarities between keyword candidates and text
# pycountry
import pycountry
# contractions
import contractions
# spacy - library for advanced natural languages processing
import spacy 
os.system('python -m spacy download en') # used for keywords filling
nlp = spacy.load("en_core_web_sm") 
os.system('python -m spacy download xx_ent_wiki_sm') # used for locations filling
nlp_wk = spacy.load("xx_ent_wiki_sm")
# sentenceTransformer
from sentence_transformers import SentenceTransformer # For sentence embeddings
# import spellchecker
from spellchecker import SpellChecker
# import tensorflow, tensforflow_hub, keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
os.environ['TFHUB_CACHE_DIR'] = f'{str(Path.home())}/workspace/tf_cache' # path for downloading pre traind model

In [2]:
# reading in the CSV files using the pandas read_csv function. we have also dropped the id column from the train set as we won’t need this for training the model.
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train = train.drop('id', axis=1)

In [3]:
def find_similarity(text_a, 
                    text_b, 
                    model = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1",
                    no_of_keywords=5):
    """
    
    """
    
    # load pretraind model for similarity
    embed = hub.load(model)
    tweet_embedding = embed([text_a])
    results_embeddings = embed(text_b)

    # calculate the similarity between document and results embeddings
    distances = cosine_similarity(tweet_embedding, results_embeddings)

    # get the top similar keywords
    keywords = [text_b[index] for index in distances.argsort()[0][-no_of_keywords:]]
    
    # get the indices of minimum element in numpy array
    keyword = keywords[np.where(distances == np.amin(distances))[0].tolist()[0]]
    
    return keyword

In [4]:
def text_formatting(text,
                    correct_spelling=True, 
                    remove_emojis=True, 
                    remove_stop_words=True):
    """
        Description: Apply function to clean a given text.
        
        :param tweet:
        :param correct_spelling:
        :param remove_emojis:
        :param remove_stop_words:
        
        Example: 
        input: 'Barbados #Bridgetown JAMAICA ÛÒ Two cars set ablaze: SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintende...  http://t.co/wDUEaj8Q4J'
        output: 'barbados bridgetown jamaica  two cars set ablaze santa cruz  head elizabeth police superintend'
    """ 
    
    def correct_spellings(text, spell=SpellChecker()):   
        """
            Correct the missplled words of a given tweet
        """
    
        text = text.split()
        misspelled = spell.unknown(text)
        result = map(lambda word : spell.correction(word) if word in misspelled else word, text)

        return " ".join(result)
    
    text = text.lower().strip()
    
    # remove urls
    text = re.compile(r'https?://\S+|www\.\S+').sub(r'', text)
    
    # remove html tags
    text = re.compile(r'<.*?>').sub(r'', text)
   
    # using contractions.fix to expand the shotened words
    text = contractions.fix(text).lower().strip()
        
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # check for spelling errors
    if correct_spelling: 
        text = correct_spellings(text)
    
    # remove emojis if any
    if remove_emojis:
        text = text.encode('ascii', 'ignore').decode('utf8').strip()
    
    # remove spaces
    text = re.sub(' +', ' ', text)

    # remove stop words (Examples of stop words in English are “a”, “the”, “is”, “are” and etc)
    if remove_stop_words:
        text = ' '.join([word for word in text.split(' ') if word not in nlp.Defaults.stop_words])
    print(text)
    return text

In [5]:
def extract_keyword(text,
                    keywords):
    """
        Description: Apply function to extract a keyword from a given text
        
        :param tweet:
        :param keywords: 
        :param model:
        :param no_of_keywords:
        
        Example: 
        input: ''
        output: ''
    """ 
    keyword = ""

    # check first if any word from the given text is already defined as a keyword somewhere else
    for k in text.split(' '):
        if k in keywords:
            keyword = k
    
    if not keyword:
        text_ = nlp(text)

        # custom list of parts-of-speech (pos) tags we are interested in
        pos_tag = ['VERB', 'NOUN", "ADJ", "PROPN']
        result = []

        # if the token pos tag matches one of the pos_tag, then add the text form of the token to result list
        for token in text_:
            if (token.pos_ in pos_tag):
                result.append(token.text)

        # find similarity between each possible keywords to the tweet itself by using a pre-traind model
        keyword = find_similarity(text, result)
    
    return keyword

In [6]:
def get_location(text):  
    """
        Description: Apply function to extract a location from a given text
        
        :param text:
        
        Example: 
        [1] input: 'tha kicks antiblight loan effort memphis'
            output: 'memphis'
        [2] input: 'mourning notices ny stabbing arson victims stir politics grief posters shira bank'
            output: 'New York'
        [3] input: 'mourning notices ny stabbing arson victims stir politics grief posters shira bank israel'
            output: 'Israel'
    """ 

    us_state_to_abbrev = {
        "Alabama": "AL",
        "Alaska": "AK",
        "Arizona": "AZ",
        "Arkansas": "AR",
        "California": "CA",
        "Colorado": "CO",
        "Connecticut": "CT",
        "Delaware": "DE",
        "Florida": "FL",
        "Georgia": "GA",
        "Hawaii": "HI",
        "Idaho": "ID",
        "Illinois": "IL",
        "Indiana": "IN",
        "Iowa": "IA",
        "Kansas": "KS",
        "Kentucky": "KY",
        "Louisiana": "LA",
        "Maine": "ME",
        "Maryland": "MD",
        "Massachusetts": "MA",
        "Michigan": "MI",
        "Minnesota": "MN",
        "Mississippi": "MS",
        "Missouri": "MO",
        "Montana": "MT",
        "Nebraska": "NE",
        "Nevada": "NV",
        "New Hampshire": "NH",
        "New Jersey": "NJ",
        "New Mexico": "NM",
        "New York": "NY",
        "North Carolina": "NC",
        "North Dakota": "ND",
        "Ohio": "OH",
        "Oklahoma": "OK",
        "Oregon": "OR",
        "Pennsylvania": "PA",
        "Rhode Island": "RI",
        "South Carolina": "SC",
        "South Dakota": "SD",
        "Tennessee": "TN",
        "Texas": "TX",
        "Utah": "UT",
        "Vermont": "VT",
        "Virginia": "VA",
        "Washington": "WA",
        "West Virginia": "WV",
        "Wisconsin": "WI",
        "Wyoming": "WY",
        "District of Columbia": "DC",
        "American Samoa": "AS",
        "Guam": "GU",
        "Northern Mariana Islands": "MP",
        "Puerto Rico": "PR",
        "United States Minor Outlying Islands": "UM",
        "U.S. Virgin Islands": "VI",
    }
    
    text_ = nlp_wk(text)
    
    loc = []
    
    for ent in text_.ents:
        if(ent.label_ == "LOC"):
            loc.append(ent.text)
            
    if not loc:
        abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))
        for item in text.upper().split(' '):
            if item in abbrev_to_us_state: 
                loc.append(abbrev_to_us_state.get(item))
    
    if len(loc) >= 2:            
        # if more then 2 locations were found, then find similarity between each possible location to the text itself by using a pre-traind model
        location = find_similarity(text, loc)
    else:
        # else convert to string
        location = ''.join(loc)
        
    return location

In [7]:
import queue
from threading import Thread, Lock


def _thread_func(q, result):
    """
    Threaded function for queue processing
    """
    while not q.empty():
        # Fetch new work from the Queue
        work = q.get()
        try:      
            data = text_formatting(f"{work[1]}")
            # Store data back at correct index
            result[work[0]] = data
        except:
            result[work[0]] = {}
        # Signal to the queue that the task has been processed
        q.task_done()
    return True

   
all_raw_tweets = []

for i in range(len(train["text"])):
    all_raw_tweets.append(train['text'].iloc[i])

# Set up a queue to hold all the tweets
q = queue.Queue(maxsize=0)

# Use many threads (50 max, or one for each file)
num_threads = min(50, len(all_raw_tweets))

# Populating Queue with tasks
tweets = [{} for x in all_raw_tweets]

# Load up the queue with the raw text to get the format vesrion of each one
for i in range(len(all_raw_tweets)):
    # Need the index and the text in each queue item
    q.put((i,f"{all_raw_tweets[i]}"))

# Starting worker threads on queue processing
for i in range(num_threads):
    worker = Thread(target=_thread_func, args=(q, tweets))
    # Setting threads as "daemon" allows main program to exit eventually even if these dont finish correctly.
    worker.setDaemon(True)
    # Start worker thread
    worker.start()

# Wait until the queue has been processed
q.join()

print(raw_text)


got sent photo ruby alaska smoke wildfires pours school
emergency evacuation happening building street
13000 people receive wildfires evacuation orders california
deeds reason earthquake allah forgive
residents asked shelter place notified officers evacuation shelter place orders expected
damage school bus 80 multi car crash breaking
hill fire woodsafraid tornado coming area
flood ago myanmar arrived ago

people died heat wave far
ridiculousflood disaster heavy rain causes flash flooding streets manitou colorado springs areas

love skiing
car fast
haha south tampa getting flooded hah wait second live south tampa going going fuck flooding
forest fire near la range ask canada
london cool
man
love fruits
wayi eat shit
like pasta
ine office asia set ablaze
end
ny week
summer lovely
love girlfriend
rockyfire update california 20 closed directions lake county fire afire wildfires
wanted set chicago ablaze preaching hotel
gained 3 followers week know stats grow
try bring heavy metal
wonderful

heard church leaders kenya coming forward comment accident issue disciplinary measuresarrestpastorngangaalexalltimelow aww airplane accident going die cuties good job

omg horrible accident man died wings airplane
mickinyman theatlantic killed airplane accident night car wreck politics best
airplane accident
320 ir iceman aftershock djicemoon dubstep trapmusic dna ed dance ices
phone looks like car ship airplane accident terrible
shooting airplane accidentairplane crashes house colombia 12 people die accident

statistically risk getting killed cop dying airplane accident
feared killed pakistani air ambulance helicopter crash
drone airplane accident pilots worried use drones esp close vicinity airports
votejkt48id mbataweel rip unladen family members killed airplanes accident
leading emergency services boss welcomes new ambulance charity
feared killed pakistani air ambulance helicopter crash
new nanotech device able target destroy blood clotsomg believe rip bro airplane accident jetengi

short reading apocalypse 211023 spirit angel took enormous high mountainzonewolf123 liked youtube video minecraft night lucky block mod bob apocalypse wither 20 amp

ourmothermary short reading apocalypse 211023 spirit angel took enormous high mountain
minecraft night lucky block mod bob apocalypse wither 20 amp mod showcase popularmmos youtube
alexandrapullin apocalypse comes week know
red faction armageddon microsoft xbox 360 2011 read ebay
enjoyed liveaction attack titan time posters reminded freshly clean coiffed apocalypse
best movie seen armageddon
fittscott minecraft night lucky block mod bob apocalypse wither 20 amp mod showcase popularmmos view
oviedo peace love amp armageddon
long coat hand worn certainty armageddon bears sense occasion
bed time wake revolution armageddon start
going beat armageddon su hao got flawless try
hey az sign petition save wildhorses tantonationalforest rollingstone signaling order
phone spying hidden door nsa data mining software financial armageddo

alleged east bay serial arsonist arrested sanfrancisco
trusting iran stop terrorism like inviting arsonist join fire brigade telegraph
arsonists blamed blaze plastics recycling business adelaide pcaldicott7 reports 7newsadl
big burning true story arsonist missing girl
doofus diamorfiend join moves
delhi government provide free treatment acid attack victims private hospitals
scares new versions nuclear attack warnings like knowing governments prepare
israeli forces raid home alleged car attack suspect palestineliked youtube video slimebeast town salem win arsonistlocalarsonist diamorfiend legal system forgets


demi stans think heart attack sold 56 million copiessuspect latest theater attack psychological issues

spotlight paradise arsonist wniagospel arsonistmusiclocalarsonist guess shit thinking

breaking terror attack police post dhampirbreaking obama officials gave muslim terrorist weapon texas attack

dattomm funniest twitter feminists try attack head
localarsonist lao93 blasts acc

detroit's interested win battle
fedex longer transport bioterror germs wake anthrax lab mishaps news phone apple mobile
happens battle block cbsbigbrother finally
fedex willing transport research specimens potential bioterror pathogens wake anthrax lab mishaps
ahl responds fedex longer transport bioterror germs wake anthrax lab mishaps
fedex stop transporting bioterror germs lab mishaps fedex stopped transporting certain research
fedex longer transport bioterror germs wake anthrax lab mishaps usatoday
world fedex longer transport bioterror germs wake anthrax lab mishaps
fedex longer transport bioterror germs usatoday
world fedex longer transport bioterror germs wake anthrax lab mishaps
kelly tomlinson mildmannered baseman great metropolitan team fights neverending battle hits fbi giants way
miss gary buses son plays dixie electronic green fiddle postbattle celebration sequence
sexydragonmagic come realization attention span mass battle games painting playing
house energy amparo commerc

...

obama declares disaster typhoondevastated saipan obama signs disaster declaration northern martians
4yygb mhtw4fnet thousands evacuated taiwan prepares strongest typhoon 2015 abc online
gym satellite bullseye typhoon soudelor
map typhoon soudelors predicted path approaches taiwan expected landfall southern china
flyer upheaval
foxes let report people instead obama declares cami federal disaster area post typhoon soudelor guys 2slow2report
global precipitation measurement satellite captures ad image typhoon soudelor nasahurricane
medieval upheaval hardy boys secret files book 18 franklin dixon aladdin kindle edition
photos typhoon soudelor aim set taiwan andchina
typhoon soudelor taking dead aim taiwan
obama declares disaster typhoondevastated saipan obama signs disaster declaration northern martians
obama declares disaster typhoondevastated saipan obama signs disaster declaration northern martians
banes soudelor typhoon soudelor taking dead aim taiwan expected landfall friday according

volcanotornado live da mill bad son perspective
people bake turning kitchen war zone eggs flour
uses 19 volcano hawaii time20150806 010401 utc20150805 150401 1000 epicenter
sundays football seasonfrom 9 11 women log ofshit complete war zone
mrmikeeaton muazimusprime hill hill mountain volcano hell mountain hill hil
mama hugging world war zone homies inmate dead wrong
allow reiterate weapon mindset individual professional help legion
honey angel like scream words weapon ahead best shot woman want leave
kasiakosek drive sucks case know desk looks like war zone goes need mode
sitting cityofmemphis traffic like sitting war zone police carefur leather coats sprite amp weapon choice lifestyle chosen 02

looks like war zone outside going
abe government said missiles weapon jade provide ally collective self defense right exercised
turned jasmines house war zone littlewomenla
uploading videos soon possible guys new weapon types action
direction concert war zone otrametlife
fuck nuclear weapon
g

throwback love tomorrow lucymayofficial whirlwindprayers cousin california helping wild fires

wild fires west crazy
man right help wild fires
wild fires freak fuck like hell
collection article video west coast wild fires abc
god forbid true california problems severe drought amp wild fires nuclear power sanonofre
load shedding permanently live like wild camp fires cook fire etc
firefighters connecticut headed california fight wild fires
map shows california wildfires burning map created cal fire
europe nature kind tornados hurricanes wild fires earthquakes flash floods mega snow droughts
kirstiealley animaladvocate pay photo safari weapons allowed otherwisethey like kill
love surely come find like blazing wild fires singing
fringe highly recommend m00nf00l titania summerhallery whirlwind reimagining shakespeare midsummer
pitches im thank toronto whirlwind amazingness glad finally
senfeinstein thanks sen einstein hurry home california huge uncontrolled wild fire lightening starting new

precious olive tree lost battleanother crazy windstorm weathernetwork
rangerkaitimay major windstorm thunder rain near jacksoncalm
chrises average gop voter biggest circus hailstormwindstorm
police officer wounded suspect dead exchanging shots
abc news police officer wounded suspect dead exchanging shots news
blakeshelton fart windstormfollow jeez
officer wounded suspect killed exchange gunfire richmond police officer wounded
asterpuppet wounded carried brothers sisters entered air ship academia
police officer wounded suspect dead exchanging shots richmond police officer wounded suspect killed
prince phillip said numbers murdered british amritsar thats bit exaggerated include wounded
like rubbing salt wounds dad
mirrors reflected private wounds desires
wocowae officer wounded suspect killed exchange gunfire
equally scared started heal fill wounds thought permanent
wop serving ask veterans ask offof vets physical wounds invisible ones client
crack path wiped morning beach run surface wo

wrecked stomach helptwins pitchers ego wrecked

wrecked world
coleslaw wrecked
wrecked tired going asleep 3300k exotic car wrecked train accident

wrecked emotions
cramer tigers 3 words wrecked disney stock
wreckage conclusively confirmed missing flight mh370 yahoonewsdigestawesome time gettin wrecked bowling night

bright wrecked
tuneswgg lol got wrecked
cramer 3 words wrecked dis stock
marynmck adorable hope wrecked noticedwrecked

cramer tigers 3 words wrecked disney stock cab tones
wreckage mh370 naib mh370 najibrazak malaysiaairlines
riddler best earlyexit primary presidential wannabe certain chances gets wrecked rich guy
days work pretty wrecked hahaha shoutout family
cramer igers 3 words wrecked disneys stock cnbc
pic yr old pak suicide bomber detonated bomb turkey army trench released
fix fore trading cramer tigers 3 words wrecked disney stock
james hustler bad architecture wrecked cities residualincome momwidda16 gone relax thought wife wrecked cake goner mind lol whoops

calg

In [8]:
print(len(raw_text))

7613


In [ ]:
# Preprocess data:
# Steps:
# [1] Format tweet: correct spelling, remove emojis, and remove stop_words
# [2] Format keyword: fill missing keywords for certain tweets following specific scenarios
# [3] Format location: fill missing locations for certain tweets following specific scenarios

# get all available keywords from the the data (unique values) for step [2]
train_keywords = train['keyword'].unique()

# for i in tqdm(range(len(train["text"]))):
for i in range(len(train["text"])):
    # Step 1 - tweet text formatting
    train['text'].iloc[i] = text_formatting(train['text'].iloc[i])
    # Step 2 - keyword formatting
    if pd.isnull(train['keyword'].iloc[i]):
        # keyword is empty, so search for a keyword within the tweet itself, if no keyword is found then fill with NaN
        try:
            train['keyword'].iloc[i] = extract_keyword(tweet=train['text'].iloc[i], keywords=train_keywords)
        except:
            train['keyword'].iloc[i] = "NaN"   
    # Step 3 - location formatting
    if pd.isnull(train['location'].iloc[i]):
        # location is empty, so search for a location within the tweet itself, if no location is found then fill with NaN 
        location = get_location(train['text'].iloc[i])
        if location:
            # location found within the tweet
            train['location'].iloc[i] = location
        else:
            # location is not found within the tweet
            train['location'].iloc[i] = "NaN"
    else:
        # location is not empty, so first make sure there's no a location within the tweet itself
        location = get_location(train['text'].iloc[i])
        if location:
            # location found within the tweet, so overwrite the value under location with it
            train['location'].iloc[i] = location
        else:
            # location is not found within the tweet, so make sure the given location is a legit one and not some garbage text
            location = get_location(train['location'].iloc[i])
            if not location:
                # location is not legit, replace it with NaN
                train['location'].iloc[i] = "NaN" 

In [ ]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2",
                           input_shape=[], dtype=tf.string)

model = keras.Sequential()
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
# split the train data into a training and test set
X = train.drop('target', axis=1)
y = train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
### LR
# The below code creates a logistic regression model that performs the defined transformations before fitting or predicting.
lr = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(solver='lbfgs'))])
# model score: 0.731
    
### KNN
# from sklearn.neighbors import KNeighborsClassifier
# lr = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', KNeighborsClassifier(3))])
# model score: 0.668

### DecisionTreeClassifier
# from sklearn.tree import DecisionTreeClassifier
# lr = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', DecisionTreeClassifier(max_depth=5))])
# model score: 0.587

### RandomForestClassifier
# from sklearn.ensemble import RandomForestClassifier
# lr = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1))])
# model score: 0.563

### SVC
# from sklearn.svm import SVC
# lr = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', SVC(kernel="linear", C=0.025))])
# model score: 0.582

### LinearDiscriminantAnalysis
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# lr = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto'))])

In [ ]:
lr.fit(X_train, y_train)
print("model score: %.3f" % lr.score(X_test, y_test))

In [ ]:
test_no_id = test.drop('id', axis=1)
test_predictions = lr.predict(test_no_id)

In [ ]:
tweet_id = test['id']
submission_df_1 = pd.DataFrame({
                  "id": tweet_id, 
                  "target": test_predictions})

In [ ]:
submission_df_1.to_csv('submission_1.csv', index=False)